# Conversation Language Understanding Solution

## Install Library

In [ ]:
%pip install azure-ai-language-conversations

## Load Azure Configurations

In [1]:
import os

# Load Azure configurations from environment variables
# Ensure that AZURE_AI_LANGUAGE_KEY and AZURE_AI_LANGUAGE_ENDPOINT are set in your environment
language_key = os.environ.get('AZURE_AI_LANGUAGE_KEY')
language_endpoint = os.environ.get('AZURE_AI_LANGUAGE_ENDPOINT')

# Represents the Project Name
project_name = "CLU_Challenge"
# Represents the output model label.
modelLabel = "Travel_CLU"  

## Create ConversationAuthoringClient

In [10]:
from azure.ai.language.conversations.authoring import ConversationAuthoringClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using Azure Key and Endpoint
def authenticate_client():
    """
    Authenticates the Azure ConversationAuthoringClient using the provided key and endpoint.

    Returns:
        ConversationAuthoringClient: An authenticated client for Azure Conversation Authoring.
    """
    credential = AzureKeyCredential(language_key)
    client = ConversationAuthoringClient(
        endpoint=language_endpoint,
        credential=credential
    )
    return client

# Initialize the client
conversation_authoring_client = authenticate_client()

## Create a Project

In [16]:
project = {
        "language": "en-us",  # The project language. This is BCP-47 representation of a language. For example, use "en" for English, "en-gb" for English (UK), "es" for Spanish etc. Required.
        "projectKind": "Conversation",  # Represents the project kind. Required. Known values are: "Conversation" and "Orchestration".
        "projectName": project_name,  # The new project name. Required.
        "description": "Sample project made in Python",  # Optional. The project description.
        "multilingual": False,  # Optional. Whether the project would be used for multiple languages or not.
        "settings": {
           "confidenceThreshold": 0.0,  # The threshold of the intent with the highest confidence, at which the prediction will automatically be changed to None. The value of the threshold should be between 0 and 1 inclusive. Required.
       }
   }

conversation_authoring_client.create_project(project_name, project)

{'createdDateTime': '2025-04-15T05:10:35Z',
 'lastModifiedDateTime': '2025-04-15T05:10:35Z',
 'projectKind': 'Conversation',
 'settings': {'confidenceThreshold': 0.0,
  'normalizeCasing': False,
  'augmentDiacritics': False},
 'projectName': 'CLU_Challenge',
 'multilingual': False,
 'description': 'Sample project made in Python',
 'language': 'en-us'}

## Define Project Assets: Intents, Entities, and Utterances

Must have 15 Utterances per Intent

In [17]:
import json

# Filepath to the JSON file
file = "exported_challenge_data.json"

# Open and load the JSON file
with open(file, "r") as json_file:
    project_assets = json.load(json_file)

## Import Project

In [18]:
poller = conversation_authoring_client.begin_import_project(
    project_name=project_name,
    project=project_assets,
)
response = poller.result()
print(response)

{'jobId': 'b660ed86-b291-4097-a21e-8ef29822258f_638802720000000000', 'createdDateTime': '2025-04-15T05:10:47Z', 'lastUpdatedDateTime': '2025-04-15T05:10:47Z', 'expirationDateTime': '2025-04-22T05:10:47Z', 'status': 'succeeded'}


## Train Model

In [19]:
# JSON input template you can fill out and use as your body input.
configuration = {
    "modelLabel": modelLabel,  # Represents the output model label. Required.
    "trainingMode": "standard",  # Represents the mode of the training operation. Required. Known values are: "advanced" and "standard".
    "evaluationOptions": {
        "kind": "manual",  # Optional. Represents the evaluation kind. By default, the evaluation kind is set to percentage. Known values are: "percentage" and "manual".
        # "testingSplitPercentage": 0,  # Optional. Represents the testing dataset split percentage. Only needed in case the evaluation kind is percentage.
        # "trainingSplitPercentage": 0  # Optional. Represents the training dataset split percentage. Only needed in case the evaluation kind is percentage.
    },
    #"trainingConfigVersion": "str"  # Optional. Represents training config version. By default, "latest" value is used which uses the latest released training config version.
   }

poller = conversation_authoring_client.begin_train(project_name=project_name,configuration=configuration)
response = poller.result()
print(response)

{'result': {'modelLabel': 'Travel_CLU', 'trainingConfigVersion': '2022-09-01', 'trainingMode': 'standard', 'trainingStatus': {'percentComplete': 100, 'startDateTime': '2025-04-15T05:10:59.5074808Z', 'endDateTime': '2025-04-15T05:11:02.585138Z', 'status': 'succeeded'}, 'evaluationStatus': {'percentComplete': 100, 'startDateTime': '2025-04-15T05:11:03.407674Z', 'endDateTime': '2025-04-15T05:11:18.4336257Z', 'status': 'succeeded'}}, 'jobId': 'd39b297b-033f-40ac-989d-b271aa933cff_638802720000000000', 'createdDateTime': '2025-04-15T05:10:58Z', 'lastUpdatedDateTime': '2025-04-15T05:12:03Z', 'expirationDateTime': '2025-04-22T05:10:58Z', 'status': 'succeeded', 'warnings': []}


## Display Model Evaluation Summary

In [20]:
evaluation_summary = conversation_authoring_client.get_model_evaluation_summary(project_name=project_name, trained_model_label=modelLabel)
print(evaluation_summary)

{'entitiesEvaluation': {'confusionMatrix': {'Date': {'Date': {'normalizedValue': 100.0, 'rawValue': 8.0}}, 'Hotel': {'Hotel': {'normalizedValue': 76.03306, 'rawValue': 8.0}, 'Location': {'normalizedValue': 23.966942, 'rawValue': 2.521739}}, 'Location': {'Location': {'normalizedValue': 87.5, 'rawValue': 14.0}, 'Hotel': {'normalizedValue': 12.5, 'rawValue': 2.0}}, 'NumOfRooms': {'NumOfRooms': {'normalizedValue': 100.0, 'rawValue': 7.0}}, 'NumOfTickets': {'NumOfTickets': {'normalizedValue': 100.0, 'rawValue': 7.0}}, '$none': {'$none': {'normalizedValue': 96.87931, 'rawValue': 55.802475}, 'Date': {'normalizedValue': 0.7309942, 'rawValue': 0.42105263}, 'Hotel': {'normalizedValue': 1.3480393, 'rawValue': 0.7764706}, 'Location': {'normalizedValue': 1.0416669, 'rawValue': 0.6}}}, 'entities': {'Date': {'f1': 0.9411764740943909, 'precision': 0.8888888955116272, 'recall': 1.0, 'truePositiveCount': 8, 'trueNegativeCount': 0, 'falsePositiveCount': 1, 'falseNegativeCount': 0}, 'NumOfRooms': {'f1': 1

## Deploy the Trained Model

In [21]:
deployment = {
       "trainedModelLabel": modelLabel  # Represents the trained model label. Required.
   }

trained_model = conversation_authoring_client.begin_deploy_project(project_name=project_name, deployment_name=modelLabel, deployment=deployment)
response = trained_model.result()
print(response)

{'deploymentName': 'Travel_CLU', 'modelId': 'Travel_CLU-20250415T051203-fee8336c9cf145e9aac120e5feca04d0', 'lastTrainedDateTime': '2025-04-15T05:12:03.5877353Z', 'lastDeployedDateTime': '2025-04-15T05:12:19Z', 'deploymentExpirationDate': '2026-08-31', 'modelTrainingConfigVersion': '2022-09-01'}


## Making the Request


## Create a ConversationAnalysisClient

In [2]:
from azure.ai.language.conversations import ConversationAnalysisClient
from azure.core.credentials import AzureKeyCredential

credential = AzureKeyCredential(language_key)
conversation_analysis_client = ConversationAnalysisClient(
        endpoint=language_endpoint,
        credential=credential
)

## Helper function to Send requests

In [3]:
# Analyze the utterance using the deployed model
def analyze_utterance(utterance):
    """
    Analyzes the given utterance using the deployed model.

    Args:
        utterance (str): The utterance to analyze.

    Returns:
        dict: The analysis result.
    """
    result = conversation_analysis_client.analyze_conversation(
        task={
            "kind": "Conversation",
            "analysisInput": {
                "conversationItem": {
                    "participantId": "1",
                    "id": "1",
                    "modality": "text",
                    "language": "en",
                    "text": utterance
                },
                "isLoggingEnabled": False
            },
            "parameters": {
                "projectName": project_name,
                "deploymentName": modelLabel,
                "verbose": True
            }
        }
    )

    # Extract and print the prediction details
    prediction = result['result']['prediction']

    print("Query:", result['result']['query'])
    print("Top Intent:", prediction['topIntent'])
    print("Project Kind:", prediction['projectKind'])
    print("\nIntents:")
    for intent in prediction['intents']:
        print(f"  - {intent['category']}: {intent['confidenceScore']:.2f}")

    print("\nEntities:")
    for entity in prediction['entities']:
        print(f"  - {entity['category']}: '{entity['text']}' (Offset: {entity['offset']}, Length: {entity['length']}, Confidence: {entity['confidenceScore']:.2f})")

In [4]:
# Sample utterance to analyze
utterance = """
    I would like to revoke my 3 flight tickets to Sydney for June 20 
    and also revoke my reservation for 3 rooms at Park Hyatt Sydney.
    Could you please help with the revocation and provide any necessary information?
"""

analyze_utterance(utterance)

Query: 
    I would like to revoke my 3 flight tickets to Sydney for June 20 
    and also revoke my reservation for 3 rooms at Park Hyatt Sydney.
    Could you please help with the revocation and provide any necessary information?

Top Intent: Cancel
Project Kind: Conversation

Intents:
  - Cancel: 0.93
  - Book: 0.78
  - None: 0.45

Entities:
  - NumOfTickets: '3 flight tickets' (Offset: 31, Length: 16, Confidence: 1.00)
  - Location: 'Sydney' (Offset: 51, Length: 6, Confidence: 1.00)
  - Date: 'June 20' (Offset: 62, Length: 7, Confidence: 1.00)
  - NumOfRooms: '3 rooms' (Offset: 110, Length: 7, Confidence: 1.00)
  - Hotel: 'Park Hyatt Sydney' (Offset: 121, Length: 17, Confidence: 1.00)


In [10]:
# Sample utterance to analyze
utterance = "I want to travel to Paris next month"
analyze_utterance(utterance)

Query: I want to travel to Paris next month
Top Intent: None
Project Kind: Conversation

Intents:
  - None: 0.78
  - Book: 0.69
  - Cancel: 0.56

Entities:
  - Location: 'Paris' (Offset: 20, Length: 5, Confidence: 1.00)
  - Date: 'next month' (Offset: 26, Length: 10, Confidence: 1.00)


In [7]:
# Sample utterance to analyze
utterance = """
    I would like to book 2 tickets to Sydney for June 20. 
    Could you please check the availability and assist with the reservation?
"""

analyze_utterance(utterance)

Query: 
    I would like to book 2 tickets to Sydney for June 20. 
    Could you please check the availability and assist with the reservation?

Top Intent: Book
Project Kind: Conversation

Intents:
  - Book: 0.97
  - Cancel: 0.87
  - None: 0.67

Entities:
  - NumOfTickets: '2 tickets' (Offset: 26, Length: 9, Confidence: 1.00)
  - Location: 'Sydney' (Offset: 39, Length: 6, Confidence: 1.00)
  - Date: 'June 20' (Offset: 50, Length: 7, Confidence: 1.00)


In [9]:
# Sample utterance to analyze
utterance = """
    I like 2 rooms at Park Hyatt Sydney for June 20.
"""

analyze_utterance(utterance)

Query: 
    I like 2 rooms at Park Hyatt Sydney for June 20.

Top Intent: Book
Project Kind: Conversation

Intents:
  - Book: 0.90
  - Cancel: 0.69
  - None: 0.59

Entities:
  - NumOfRooms: '2 rooms' (Offset: 12, Length: 7, Confidence: 1.00)
  - Hotel: 'Park Hyatt Sydney' (Offset: 23, Length: 17, Confidence: 1.00)
  - Date: 'June 20' (Offset: 45, Length: 7, Confidence: 1.00)
